In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt

from sqlalchemy import create_engine

In [ ]:
engine = create_engine("postgresql:///kcmo-mc")
db_conn = engine.connect()

#### Race

Racial breakdown of the ~57K individuals

In [ ]:
r1 = """
with demog as (
    select 
        last_name, 
        first_name, 
        dob, 
        sex 
    from raw_court.dssgname 
    group by last_name, first_name, dob, sex
)
select 
    sex, 
    count(*) 
from demog 
group by sex;
"""
pd.read_sql(r1, db_conn)

Num cases by racial group

In [ ]:
r = """
select 
    race, 
    count(case_num) 
from raw_court.dssgname 
group by race 
order by count(*); 
"""
pd.read_sql(r, db_conn)

#### Gender
gender breakdown of the ~57K individuals

In [ ]:
g = """
with demog as (
    select 
        last_name, 
        first_name, 
        dob, 
        sex 
    from raw_court.dssgname 
    group by last_name, first_name, dob,  sex
)
select 
    sex, 
    count(*) 
from demog 
group by sex;
"""
pd.read_sql(g, db_conn)

In [ ]:
Num cases by sex

In [ ]:
g1 = """
select 
    sex, 
    count(case_num) 
from raw_court.dssgname 
group by sex 
order by count(*); 
"""
pd.read_sql(g1, db_conn)

### Zipcodes

How many zipcodes associated with each individual?

In [ ]:
q4 = """
with zipcase as (
    select 
        last_name, 
        first_name, 
        dob, 
        rcd.case_num, 
        zipcode 
    from raw_court.dssgname rcd
    inner join raw_court.dssgaddress6222022 rcda 
    on rcda.case_num = rcd.case_num
), 
demog as (
    select 
        last_name, 
        first_name, 
        dob, 
        zipcode 
    from zipcase 
    group by last_name, first_name, dob, zipcode
)
select 
    last_name, 
    first_name, 
    dob, 
    count(distinct substring(zipcode, 1, 5)) as k 
from demog 
group by last_name, first_name, dob 
order by k desc;
"""
q4case = pd.read_sql(q4, db_conn)
q4case

In [ ]:
sns.histplot(data=q4case, x="k")
# Vast majority of people have only 1-2 zipcodes associated with them.
# The max number of zipcodes any single individual has is 12.

In [ ]:
sns.histplot(data=q4case[q4case.k >= 6], x="k")

How many zipcodes in KCMO vs. not?

In [ ]:
#-- how many zipcodes fall in KCMO (71440) vs. not (18036)
z2 = """
with zipcase as (
    select 
        last_name, 
        first_name, 
        dob, 
        rcd.case_num, 
        zipcode 
    from raw_court.dssgname rcd
    inner join raw_court.dssgaddress6222022 rcda 
    on rcda.case_num = rcd.case_num
), 
demog as (
    select 
        last_name, 
        first_name, 
        dob, 
        zipcode 
    from zipcase 
    group by last_name, first_name, dob, zipcode
),
kc as (
    select 
        case when substring(zipcode, 1, 2) = '64' then 1 else 0 end as kcmo 
    from demog
)
select 
    kcmo, 
    count(*) 
from kc 
group by kcmo;
"""
pd.read_sql(z2, db_conn)

Num distinct cases by zip codes

In [ ]:
#--counts of zip codes, extract first 5 digits, limit to top 15 most commonly occurring
z1 = """
with zipprob as (
    select 
        last_name, 
        first_name, 
        dob, 
        race, 
        rcd.case_num, 
        substring(zipcode, 1, 5) as zip 
    from raw_court.dssgname rcd 
    inner join raw_court.dssgaddress6222022 rcda 
    on rcda.case_num = rcd.case_num
)
select 
    zip, 
    count(distinct case_num) 
from zipprob 
group by zip 
order by count(*) 
desc limit 15;
"""
pd.read_sql(z1, db_conn)

#### Num indv. of certain races in each zipcode
---neighborhood racial percentages using census data can be compared with this to see if disparate impact

In [ ]:
q5 = """
with zipprob as (
    select 
    last_name, 
    first_name, 
    dob, 
    race, 
    rcd.case_num, 
    substring(zipcode, 1, 5) as zip 
    from raw_court.dssgname rcd 
    inner join raw_court.dssgaddress6222022 rcda 
    on rcda.case_num = rcd.case_num
)
select 
    zip, 
    race, 
    count (*) 
from zipprob 
group by zip, race 
order by count(*) desc;
"""
pd.read_sql(q5, db_conn)

#### In each zipcode, 

In [ ]:
#--zipcodes first 5 digits grouped by sex
q6 = """
with zipprob as (
    select 
        last_name, 
        first_name, 
        dob, 
        sex, 
        rcd.case_num, 
        substring(zipcode, 1, 5) as zip 
    from raw_court.dssgname rcd 
    inner join raw_court.dssgaddress6222022 rcda 
    on rcda.case_num = rcd.case_num)
select 
    zip, 
    sex, 
    count(*) 
from zipprob 
group by zip, sex 
order by count(*) desc;
"""
pd.read_sql(q6, db_conn)

### How many probations (probation terms) each year?

In [ ]:
#count_all: number of probation terms starting in each year (adds up to 173930),
#count_distinct_cases: number of probation terms starting in each year associated with 
# distinct case numbers (adds up to 115642). If we were to group by start_year, case_num
# the number of cases across the 10 years would add up to 114,820. 
q7 = """
select 
    extract(year from start_dttm::timestamp::date) as start_year,
    count(*) as count_all, 
    count(distinct case_num) as count_distinct_cases
from raw_court.dssgc_ordv6222022 
group by start_year; 
"""
pd.read_sql(q7, db_conn)

In [ ]:
#-- number of probation terms with a start date in each year, grouped by defendants.
# adds up to ~79k. Unique indv. may have multiple probation terms, hence doesn't add up to 57K
q8 = """
with ordv_name as (
    select * 
    from raw_court.dssgc_ordv6222022 do2  
    inner join raw_court.dssgname n 
    on do2.case_num = n.case_num
),
ordv_name2 as (
    select 
        last_name, 
        first_name, 
        dob, 
        start_dttm 
    from ordv_name 
    group by last_name, first_name, dob, start_dttm)
select 
    extract(year from start_dttm::timestamp::date) as start_year, 
    count(*)
from ordv_name2 
group by start_year 
order by start_year;
"""
pd.read_sql(q8, db_conn)

## Which probation terms were most common in each year?

In [ ]:
q9 = """
select 
    extract(year from start_dttm::timestamp::date) as start_year, 
    prob_code, 
    count(*) as count
from raw_court.dssgc_ordv6222022 do2 
group by start_year, prob_code 
order by count 
desc limit 30;
"""
pd.read_sql(q9, db_conn)
#--most common is PROB for most years, followed by:
#-- DNDMV stands for DO NOT DRIVE A MOTOR VEHICLE UNLESS LICENSED AND INSURED
#-- DNOOFF Do Not Obtain Any Similar Offenses
#-- CMSR20	Community Service-20 hours
#-- DIP4HR	Driver Improvement-4 hour

In [ ]:
q10 = """
select 
    extract(year from start_dttm::timestamp::date) as start_year, 
    prob_code, 
    count(*) as count
from raw_court.dssgc_ordv6222022 do2 
group by start_year, prob_code 
order by start_year, prob_code;
"""
pd.read_sql(q10, db_conn)

## How many cases per individual?

In [ ]:
# group by unique individuals and count distinct cases for each indv.
q1 = """
select 
    count(distinct case_num) as num_cases
from raw_court.dssgname 
group by last_name, first_name, dob
order by num_cases desc;
"""
cases = pd.read_sql(q1, db_conn)
cases

In [ ]:
sns.distplot(cases.num_cases, color="purple", bins=30, kde=False)

The majority of defendants have less than 5-6 cases, but goes up to 39 for one individual. 

In [ ]:
#-- How many dispositions each year?
q3 = """
with cases as (
    select * 
    from raw_court.dssgc_ordv6222022 do2
    inner join raw_court.dssgcharge d 
    on do2.case_num = d.case_num
)
select 
    count(*), 
    extract(year from disp_date::timestamp::date) as disp_year 
from cases 
group by disp_year 
order by disp_year desc;
"""
numdispyears = pd.read_sql(q3, db_conn)
numdispyears

In [ ]:
# yearly number of dispositions
yearsplot = sns.barplot(data = numdispyears, x = "disp_year", y = "count")
yearsplot.set_xticklabels(yearsplot.get_xticklabels(), rotation = 45)
yearsplot

2015 and 2019 had the highest numbers of dispositions. Significant drop in 2020, 21 due to Covid.

### Dispositions

In [ ]:
#--Disposition codes distribution 2 ROW
disp = """
with cases as (
    select * 
    from raw_court.dssgchrg_prob6222022 dp 
    inner join raw_court.codes_fixed cf 
    on dp.disp_code = cf.code
    where code_type = 'OSCAIMCD')
select 
    disp_code, 
    code_desc, 
    count(distinct case_num) 
from cases 
group by disp_code, code_desc 
order by COUNT(*) desc;
"""
pd.read_sql(disp, db_conn)

### Pleas

In [ ]:
p1 = """
with cases as (
    select * 
    from raw_court.dssgcharge dc
    inner join raw_court.codes_fixed cf 
    on dc.plea = cf.code
    where code_type = 'PLEA')
select 
    plea, 
    code_desc, 
    count(distinct case_num) 
from cases
group by plea, code_desc 
order by count(*) desc;
"""
pd.read_sql(p1, db_conn)

### SIS/SES and other types

In [ ]:
# count probation terms - SIS/SES combinations, with description of codes
# where clause and inner join on subset of codes_fixed to only display relavant rows
sent_prob = """
with cases as (
    select * 
    from raw_court.dssgc_ordv6222022 do2
    inner join raw_court.dssgcharge d
    on do2.case_num = d.case_num
), 
sis_ses as (
    select 
        prob_code, 
        sent_exec_code, 
        count(*) as numcases 
    from cases 
    group by prob_code, sent_exec_code
),
codes as (
    select 
        code_type, 
        code, 
        code_desc 
    from raw_court.codes_fixed 
    group by code_type, code, code_desc
)
select 
    prob_code, 
    sent_exec_code, 
    numcases, 
    code_desc 
from sis_ses 
inner join codes 
on sis_ses.prob_code = codes.code 
where code_type = 'PROBCNDS' 
order by numcases 
desc limit 15;
"""
pd.read_sql(sent_prob, db_conn)

In [ ]:
s1 = """
with chrg as (
    select * 
    from raw_court.dssgcharge do3 
    inner join raw_court.codes_fixed cf 
    on cf.code = do3.sent_exec_code
)
select 
    sent_exec_code, 
    count(*), 
    code_desc from chrg
group by sent_exec_code, code_desc 
order by count(*) desc;
"""
pd.read_sql(s1, db_conn) ### needs to be updated using CODE_TYPE from codes_fixed to only have one row per code

In [ ]:
#Counts of cases by SIS/SES and prob codes
s2 = """
with cases as (
    select * 
    from raw_court.dssgc_ordv6222022 do2
    inner join raw_court.dssgcharge d 
    on do2.case_num=d.case_num
)
select 
    sent_exec_code, 
    prob_code, 
    count(*)
from cases
group by sent_exec_code, prob_code 
order by count(*) desc;
"""
cases_sis_ses = pd.read_sql(s2, db_conn)
cases_sis_ses

In [ ]:
#-- How many dispositions each year?
q_sent = """
with sent as (select 
	 sent_exec_code, 
	 extract(year from disp_date::timestamp::date) as start_year
	--code_desc,
from raw_court.dssgcharge d 
inner join raw_court.codes_fixed cf 
on d.sent_exec_code = cf.code)
select 
	sent_exec_code, 
	start_year, 
	count(*) from sent 
where start_year > '2011' 
group by start_year, sent_exec_code 
order by start_year, sent_exec_code
"""
sent1 = pd.read_sql(q_sent, db_conn)
sent1

In [ ]:
# yearly dist. of sentence type
s_pl = sns.barplot(data = sent1, x = "start_year", y = "count", hue="sent_exec_code")
s_pl.set_xticklabels(s_pl.get_xticklabels(), rotation = 45)

In [ ]:
s3 = """
with j as (
    select 
        id.dssg_id,
        d.case_num, 
        sent_exec_code,
        disp_date
    from clean.dispositions d
    inner join clean.id_with_cases id
    on d.case_num = id.case_num
    where disp_date > '2011-12-31'
)
select 
    sent_exec_code, 
    count(distinct dssg_id) counts 
from j
group by sent_exec_code 
order by counts desc;
"""
pd.read_sql(s3, db_conn)

In [ ]:
s3b = """
with j as (
    select 
        id.dssg_id,
        d.case_num, 
        sent_exec_code,
        disp_date
    from clean.dispositions d
    inner join clean.id_with_cases id
    on d.case_num = id.case_num
    where disp_date > '2011-12-31'
)
select 
    sent_exec_code, 
    count(distinct dssg_id) counts 
from j
group by sent_exec_code 
order by counts desc;
"""
pd.read_sql(s3b, db_conn)

In [ ]:
s4 = """
with j as (
    select 
        disp_date,
        id.dssg_id,
        d.case_num, 
        sent_exec_code,
        extract(year from disp_date::timestamp::date) as disp_year 
    from clean.dispositions d
    inner join clean.id_with_cases id
    on d.case_num = id.case_num
)
select 
    disp_year, 
    count(distinct case_num) as distinct_cases, 
    count(distinct dssg_id) as distinct_indv
from j
where sent_exec_code = 'SIS' 
and disp_date > '2011-12-31'
group by disp_year
order by disp_year desc; 
"""
sis_per_year_uniqueid = pd.read_sql(s4, db_conn)
sis_per_year_uniqueid

In [ ]:
total = """ 
select 
    count (distinct dssg_id) 
from clean.id_with_cases
"""
pd.read_sql(total, db_conn)

In [ ]:
q_SIS_2012 = """ 
with cases as (
select 
    person_id,
    disp_date,
    sent_exec_code,
    case_num
from clean.dispositions
where sent_exec_code = 'SIS'
and disp_date > '2011-12-31'
)
select 
    count(distinct person_id) as distinct_indv,
    count(distinct case_num) as distinct_cases
from cases
"""
pd.read_sql(q_SIS_2012, db_conn) 
# 47104 unique individuals with SIS cases with disp dates beginning in 2012

In [ ]:
q_probs_sis = """ 
with cases as (
select 
    person_id,
    disp_date,
    sent_exec_code,
    case_num
from clean.dispositions
where sent_exec_code = 'SIS'
and disp_date > '2011-12-31'
),
probs as (
select 
    cases.*, 
    final_action, 
    prob_code 
from cases
left join raw_court.dssgc_ordv6222022 ov
on ov.case_num = cases.case_num
), 
wcodes as (
    select * from probs
    left join raw_court.codes_fixed cf
    on probs.prob_code = cf.code
    where code_type = 'PROBCNDS'
)
select
    prob_code,
    code_desc,
    count(distinct case_num),
    round(count(distinct case_num)/74961.0, 2) as pr
from wcodes
group by prob_code, code_desc
order by count desc 
limit 30;
"""
probs_sis = pd.read_sql(q_probs_sis, db_conn) 
probs_sis
#--most common is PROB for most years, followed by:
#-- DNDMV stands for DO NOT DRIVE A MOTOR VEHICLE UNLESS LICENSED AND INSURED
#-- DNOOFF Do Not Obtain Any Similar Offenses
#-- CMSR20	Community Service-20 hours
#-- DIP4HR	Driver Improvement-4 hour

In [ ]:
 probs_sis['pr'].sum() 
# makes sense that it doesn't add to 1 as cases receive multiple probation terms

In [ ]:
distinct_cases=sns.barplot(data = sis_per_year_uniqueid, x = "disp_year", y = "distinct_cases")
distinct_cases.set_xticklabels(distinct_cases.get_xticklabels(), rotation = 45)
distinct_cases.set_xlabel("disposition year")
distinct_cases.set_ylabel("count")
distinct_cases.set_title("Number of distinct cases falling under SIS categorization by year")
distinct_cases

In [ ]:
distinct_indv=sns.barplot(data = sis_per_year_uniqueid, x = "disp_year", y = "distinct_indv")
distinct_indv.set_xticklabels(distinct_cases.get_xticklabels(), rotation = 45)
distinct_indv.set_xlabel("disposition year")
distinct_indv.set_ylabel("count")
distinct_indv.set_title("Number of individuals receiving SIS sentences by year")
distinct_indv

In [ ]:
s5 = """
with yearly_sis as (
    select 
        id.dssg_id,
        d.case_num, 
       sent_exec_code,
        disp_date
    from clean.dispositions d
    inner join clean.id_with_cases id
    on d.case_num = id.case_num
),
outcomes as (
    select yearly_sis.*, ord.final_action 
    from yearly_sis
    inner join raw_court.dssgc_ordv6222022 ord
    on yearly_sis.case_num = ord.case_num
)
select sent_exec_code, final_action, count (distinct dssg_id)
from outcomes
where sent_exec_code = 'SIS' or sent_exec_code = 'SES' 
and disp_date > '2011-12-31'
group by sent_exec_code, final_action; 
"""
sis_ses_compl_yearly_id = pd.read_sql(s5, db_conn)
sis_ses_compl_yearly_id

In [ ]:
s6 = """
with yearly_sis as (
    select 
        id.dssg_id,
        d.case_num, 
        sent_exec_code,
        extract(year from disp_date::timestamp::date) as disp_year 
    from clean.dispositions d
    inner join clean.id_with_cases id
    on d.case_num = id.case_num
),
outcomes as (
    select yearly_sis.*, ord.final_action 
    from yearly_sis
    inner join raw_court.dssgc_ordv6222022 ord
    on yearly_sis.case_num = ord.case_num
), 
c as (
    select 
        outcomes.*, cod.code_desc, cod.code_type
    from outcomes
    inner join raw_court.codes_fixed cod
    on outcomes.final_action = cod.code
)
select 
    final_action,
    code_desc, 
    count (distinct case_num), 
    round(count (distinct case_num)/74961.0, 2) as prop
from c
where sent_exec_code = 'SIS' 
and code_type = 'PROBFACT'
--and disp_year > '2011'
group by final_action, code_desc
order by count desc; 
"""
sis_compl_yearly_id = pd.read_sql(s6, db_conn)
sis_compl_yearly_id


In [ ]:
s7 = """
with yearly_sis as (
    select 
        id.dssg_id,
        d.case_num, 
        sent_exec_code,
        extract(year from disp_date::timestamp::date) as disp_year 
    from clean.dispositions d
    inner join clean.id_with_cases id
    on d.case_num = id.case_num
),
outcomes as (
    select yearly_sis.*, ord.final_action 
    from yearly_sis
    inner join raw_court.dssgc_ordv6222022 ord
    on yearly_sis.case_num = ord.case_num
), 
c as (
    select 
        outcomes.*, cod.code_desc, cod.code_type
    from outcomes
    inner join raw_court.codes_fixed cod
    on outcomes.final_action = cod.code
)
select final_action, code_desc, count (distinct dssg_id), round(count (distinct dssg_id)/15255.0, 2) as prop
from c
where sent_exec_code = 'SES' 
and code_type = 'PROBFACT'
group by final_action, code_desc
order by count desc; 
"""
ses_compl_yearly_id = pd.read_sql(s6, db_conn)
ses_compl_yearly_id

In [ ]:
# case when statement for sis with or w/o conditions
# get proportions of completion

In [ ]:
# Community service outcomes
q_cs_outcomes = """ 
with cases as (
select 
    person_id,
    disp_date,
    sent_exec_code,
    case_num
from clean.dispositions
where sent_exec_code = 'SIS'
and disp_date > '2011-12-31'
),
probs as (
select 
    cases.*, 
    final_action, 
    prob_code 
from cases
left join raw_court.dssgc_ordv6222022 ov
on ov.case_num = cases.case_num
), 
wcodes as (
    select * from probs
    left join raw_court.codes_fixed cf
    on probs.prob_code = cf.code
    where code_type = 'PROBCNDS'
)
select
    prob_code as probation_code,
    --code_desc as description,
    final_action,
    count(distinct case_num)
from wcodes
where prob_code ilike '%%CMS%%'
and final_action= 'CMPL'
group by prob_code, final_action
order by prob_code, final_action
"""
cs_outcomes_by_code = pd.read_sql(q_cs_outcomes, db_conn)
#cs_percent_complete = pd.DataFrame(round(cs_outcomes_by_code['count'] / cs_type_counts['count'].values,2))
#cs_percent_complete.rename(columns={'count':'percent complete'}, inplace=True)
#cs_percent_complete